In [1]:
import collections
import copy
import datetime
import itertools
import sys

import json
import pathlib
import re
from pprint import pprint

import nest_asyncio
nest_asyncio.apply()
import texttable

import pytourney
import pyfumbbl
import cibblbibbl

In [2]:
show_Ids = True
G = cibblbibbl.CIBBL
Achievement = cibblbibbl.achievement.Achievement
Team = cibblbibbl.team.Team
player = cibblbibbl.player.player
Match = cibblbibbl.match.Match
Matchup = cibblbibbl.matchup.Matchup
Replay = cibblbibbl.replay.Replay
Tournament = cibblbibbl.tournament.handler.default.Tournament
NormalPlayer = cibblbibbl.player.NormalPlayer
StarPlayer = cibblbibbl.player.StarPlayer

In [3]:
G.init()

In [4]:
tournamentId=47987

In [5]:
T = G.tournaments[str(tournamentId)]

In [6]:
T.standings()

[{'cas': 12,
  'casdiff': 10,
  'score': 6,
  'scorediff': 3,
  'td': 6,
  'tddiff': 3,
  'pts': 6,
  'prestige': 60,
  'cto': -1,
  'hth': 1,
  'perf': [('W', 4124012), ('L', 4125626), ('W', 4141456)],
  'team': Team(854257),
  'nr': 1},
 {'cas': 3,
  'casdiff': -5,
  'score': 6,
  'scorediff': 2,
  'td': 4,
  'tddiff': 0,
  'pts': 6,
  'prestige': 60,
  'cto': -1,
  'hth': 2,
  'perf': [('L', 4124012), ('W', 4128163), ('B', None)],
  'team': Team(854678),
  'nr': 2},
 {'cas': 4,
  'casdiff': 1,
  'score': 3,
  'scorediff': -1,
  'td': 3,
  'tddiff': 1,
  'pts': 4,
  'prestige': 40,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4122504), ('W', 4125626), ('F', None)],
  'team': Team(942643),
  'nr': 3},
 {'cas': 4,
  'casdiff': -6,
  'score': 3,
  'scorediff': -4,
  'td': 3,
  'tddiff': -4,
  'pts': 1,
  'prestige': 10,
  'cto': -1,
  'hth': -1,
  'perf': [('D', 4122504), ('L', 4128163), ('L', 4141456)],
  'team': Team(925779),
  'nr': 4}]

In [8]:
T.achievements

set()

In [10]:
T.awarded

'yes'

In [8]:
T.status

'Completed'

In [21]:
self=T

In [22]:
  def standings(self):
    tpkeys = (
        "cas", "casdiff",
        "score", "scorediff",
        "td", "tddiff",
        "pts", "prestige"
    )
    template = {k: 0 for k in tpkeys}
    template["cto"] = -1
    template["hth"] = -1
    template["perf"] = []
    S = collections.defaultdict(lambda: copy.deepcopy(template))
    for Te in self.teams(True):
      S[str(Te.Id)].update({"team": Te})
    Ccto = self.config.get("cto", {})
    Chth = self.config.get("hth", {})
    Co = self.config.get("order")
    HTH_results = []
    for Mu in self.matchups:
      if Mu.excluded == "yes":
        continue
      HTH_result = {}
      for teamId, TP in Mu.config["team"].items():
        if teamId.isdecimal():
          Te = cibblbibbl.team.Team(int(teamId))
        else:
          Te = self.get_team(teamId)
        r = TP.get("r")
        if r is None:
          continue  # no result yet
        matchId = (Mu.match.Id if Mu.match else None)
        perf = r, matchId
        d = S[teamId]
        d.setdefault("team", Te)
        d["perf"].append(perf)
        for k in tpkeys:
          d[k] += TP.get(k, 0)
        if r.lower() not in ("b", "f", "x", "-", ""):
          HTH_result[teamId] = TP["td"]
      if len(HTH_result) == 2:
        HTH_results.append(HTH_result)
    bypts = collections.defaultdict(list)
    for teamId, d in S.items():
      if d["perf"]:
        bypts[d["pts"]].append(teamId)
    for pts, teamIds in bypts.items():
      if 1 < len(teamIds):
        pts_HTH_results = [{teamId: 0} for teamId in teamIds]
            # this ensures all nodes for HTH calculation
        for r0 in HTH_results:
            # generate edges by passing results of the group
            # members within the group
          r1 = {
              teamId: tds
              for teamId, tds in r0.items()
              if teamId in teamIds
          }
          if 1 < len(r1):
            # I do not want to pass empty dictionaries nor nodes
            # as those were ensured before
            pts_HTH_results.append(r1)
        hth_mname = self.config.get("hth_module", "sweep")
        hth_module = getattr(pytourney.tie, f'hth_{hth_mname}')
        pts_HTH = hth_module.calculate(pts_HTH_results)
        for teamId, hth_val in pts_HTH.items():
          S[teamId]["hth"] = Chth.get(teamId, hth_val)
    for teamId, cto_val in Ccto.items():
      S[teamId]["cto"] = cto_val
    # determine missing coin toss
    bytie = collections.defaultdict(list)
    keys = ("pts", "hth", "scorediff", "casdiff", "cto")
    for teamId, d in S.items():
      key = tuple(d[k] for k in keys) + ((not d["perf"]),)
      bytie[key].append(teamId)
    if self.status == "Completed":
      for k, teamIds in bytie.items():
        if 1 < len(teamIds):
          cto_val = -112  # indicate missing
          for teamId in teamIds:
            if not S[teamId]["perf"]:
              continue
            S[teamId]["cto"] = cto_val
            Ccto = self.config.setdefault("cto", {})
                # this ensures that I write to the config file
            Ccto[teamId] = cto_val
      cto_val = -999  # indicate no performance
      for teamId, d in S.items():
        if not d["perf"]:
          d["cto"] = cto_val
    else:
      try:
        del self.config["cto"]
      except KeyError:
        pass
    # determine order
    if Co is None:
      order = sorted(S, key=(
          lambda teamId: (
              (not S[teamId]["perf"]),
              -S[teamId]["pts"],
              +S[teamId]["hth"],
              -S[teamId]["tddiff"],
              -S[teamId]["casdiff"],
              -S[teamId]["cto"],
              S[teamId]["team"].name,
          )
      ))
    else:
      order = [str(teamId) for teamId in Co]
    S.default_factory = None
        # close defaultdict so it can raise KeyError exceptions
    L = [S[teamId] for teamId in order if teamId in S]
    prev_sortkey = None
    prev_nr = None
    for nr, d in enumerate(L, 1):
      if not d["perf"]:
        d["nr"] = None  # no performance
      else:
        sortkey = (
            d["pts"],
            d["hth"],
            d["tddiff"],
            d["casdiff"],
            d["cto"],
        )
        if sortkey == prev_sortkey:
          d["nr"] = prev_nr
        else:
          d["nr"] = prev_nr = nr
    return L

In [31]:
  def standings(self):
    tpkeys = (
        "cas", "casdiff",
        "score", "scorediff",
        "td", "tddiff",
        "pts", "prestige"
    )
    template = {k: 0 for k in tpkeys}
    template["cto"] = -1
    template["hth"] = -1
    template["perf"] = []
    S = collections.defaultdict(lambda: copy.deepcopy(template))
    for Te in self.teams(True):
      S[str(Te.Id)].update({"team": Te})
    Ccto = self.config.get("cto", {})
    Chth = self.config.get("hth", {})
    Co = self.config.get("order")
    HTH_results = []
    for Mu in self.matchups:
      if Mu.excluded == "yes":
        continue
      pprint(Mu.config)
      HTH_result = {}
      for teamId, TP in Mu.config["team"].items():
        if teamId.isdecimal():
          Te = cibblbibbl.team.Team(int(teamId))
        else:
          Te = self.get_team(teamId)
        r = TP.get("r")
        if r is None:
          continue  # no result yet
        matchId = (Mu.match.Id if Mu.match else None)
        perf = r, matchId
        d = S[teamId]
        d.setdefault("team", Te)
        d["perf"].append(perf)
        for k in tpkeys:
          d[k] += TP.get(k, 0)
        if r.lower() not in ("b", "f", "x", "-", ""):
          HTH_result[teamId] = TP["td"]
      if len(HTH_result) == 2:
        HTH_results.append(HTH_result)
    bypts = collections.defaultdict(list)
    for teamId, d in S.items():
      if d["perf"]:
        bypts[d["pts"]].append(teamId)
    for pts, teamIds in bypts.items():
      if 1 < len(teamIds):
        pts_HTH_results = [{teamId: 0} for teamId in teamIds]
            # this ensures all nodes for HTH calculation
        for r0 in HTH_results:
            # generate edges by passing results of the group
            # members within the group
          r1 = {
              teamId: tds
              for teamId, tds in r0.items()
              if teamId in teamIds
          }
          if 1 < len(r1):
            # I do not want to pass empty dictionaries nor nodes
            # as those were ensured before
            pts_HTH_results.append(r1)
        hth_mname = self.config.get("hth_module", "sweep")
        hth_module = getattr(pytourney.tie, f'hth_{hth_mname}')
        pts_HTH = hth_module.calculate(pts_HTH_results)
        for teamId, hth_val in pts_HTH.items():
          S[teamId]["hth"] = Chth.get(teamId, hth_val)
    for teamId, cto_val in Ccto.items():
      S[teamId]["cto"] = cto_val
    # determine missing coin toss
    bytie = collections.defaultdict(list)
    keys = ("pts", "hth", "scorediff", "casdiff", "cto")
    pprint(S)
    for teamId, d in S.items():
      key = tuple(d[k] for k in keys) + ((not d["perf"]),)
      bytie[key].append(teamId)
    if self.status == "Completed":
      for k, teamIds in bytie.items():
        if 1 < len(teamIds):
          cto_val = -112  # indicate missing
          for teamId in teamIds:
            if not S[teamId]["perf"]:
              continue
            S[teamId]["cto"] = cto_val
            Ccto = self.config.setdefault("cto", {})
                # this ensures that I write to the config file
            Ccto[teamId] = cto_val
      cto_val = -999  # indicate no performance
      for teamId, d in S.items():
        if not d["perf"]:
          d["cto"] = cto_val
    else:
      try:
        del self.config["cto"]
      except KeyError:
        pass
    # determine order
    if Co is None:
      order = sorted(S, key=(
          lambda teamId: (
              (not S[teamId]["perf"]),
              -S[teamId]["pts"],
              +S[teamId]["hth"],
              -S[teamId]["tddiff"],
              -S[teamId]["casdiff"],
              -S[teamId]["cto"],
              S[teamId]["team"].name,
          )
      ))
    else:
      order = [str(teamId) for teamId in Co]
    S.default_factory = None
        # close defaultdict so it can raise KeyError exceptions
    L = [S[teamId] for teamId in order if teamId in S]
    prev_sortkey = None
    prev_nr = None
    for nr, d in enumerate(L, 1):
      if not d["perf"]:
        d["nr"] = None  # no performance
      else:
        sortkey = (
            d["pts"],
            d["hth"],
            d["tddiff"],
            d["casdiff"],
            d["cto"],
        )
        if sortkey == prev_sortkey:
          d["nr"] = prev_nr
        else:
          d["nr"] = prev_nr = nr
    return L

In [32]:
standings(T)

{'!excluded': 'no',
 'player': {},
 'team': {'925779': {'cas': 0,
                     'casdiff': 0,
                     'name': 'Divine Sun',
                     'scorediff': 0,
                     'tddiff': 0},
          '942643': {'cas': 0,
                     'casdiff': 0,
                     'name': 'Rōnin Reborn',
                     'scorediff': 0,
                     'tddiff': 0}}}
{'!excluded': 'no',
 'player': {},
 'team': {'854257': {'cas': 0,
                     'casdiff': 0,
                     'name': 'Grand Celestial Delegate',
                     'scorediff': 0,
                     'tddiff': 0},
          '854678': {'cas': 0,
                     'casdiff': 0,
                     'name': 'Lords of the Silent Death',
                     'scorediff': 0,
                     'tddiff': 0}}}
{'!excluded': 'no',
 'player': {},
 'team': {'854257': {'cas': 0,
                     'casdiff': 0,
                     'name': 'Grand Celestial Delegate',
               

[{'cas': 0,
  'casdiff': 0,
  'score': 0,
  'scorediff': 0,
  'td': 0,
  'tddiff': 0,
  'pts': 0,
  'prestige': 0,
  'cto': -999,
  'hth': -1,
  'perf': [],
  'team': Team(925779),
  'nr': None},
 {'cas': 0,
  'casdiff': 0,
  'score': 0,
  'scorediff': 0,
  'td': 0,
  'tddiff': 0,
  'pts': 0,
  'prestige': 0,
  'cto': -999,
  'hth': -1,
  'perf': [],
  'team': Team(854257),
  'nr': None},
 {'cas': 0,
  'casdiff': 0,
  'score': 0,
  'scorediff': 0,
  'td': 0,
  'tddiff': 0,
  'pts': 0,
  'prestige': 0,
  'cto': -999,
  'hth': -1,
  'perf': [],
  'team': Team(854678),
  'nr': None},
 {'cas': 0,
  'casdiff': 0,
  'score': 0,
  'scorediff': 0,
  'td': 0,
  'tddiff': 0,
  'pts': 0,
  'prestige': 0,
  'cto': -999,
  'hth': -1,
  'perf': [],
  'team': Team(942643),
  'nr': None}]